#  fase 4 Encontrar asignacion del fuid

In [ ]:
import os
import pandas as pd
from openpyxl import load_workbook

# ————— CONFIGURA ESTO —————
# Ruta al Excel maestro con las columnas: nombre, cedula, depe, ser, subs
DATA_PATH      = r'C:\Users\juans\Downloads\dev_prev\resources\salida_asuntos_v2.xlsx'
# Carpeta donde están tus archivos *_hoja_control.xlsx
TEMPLATE_DIR   = r'C:\Users\juans\Documents\proarchitecg\version_2_docker\model_clasification_image_v2\answer\hojas_control'
# Carpeta donde quieres que se guarden las hojas ya completadas
OUTPUT_DIR     = r'C:\Users\juans\Downloads\dev_prev\formato final\salida'
# Nombre de la hoja dentro de cada libro (si no es la primera hoja cambia este valor)
SHEET_NAME     = None  # None = primera hoja activa
# ————————————————————————

In [ ]:
# 1) Carga el maestro y prepara la columna de matching
df = pd.read_excel(DATA_PATH, dtype={'cedula':str})
df['nombre_up'] = df['nombre'].str.upper().str.strip()

# 2) Define el mapeo de 'depe' → valores de D6,D7,D8,D9
DEPE_MAP = {
    30000: {
        'D6': 'COMISIÓN DE LA MESA',
        'D7': 'SECRETARÍA GENERAL',
        'D8': 'HISTORIAS LABORALES',
        'D9': '',
    },
    4100: {
        'D6': 'DIRECCIÓN ADMINISTRATIVA',
        'D7': 'DIVISIÓN DE PERSONAL',
        'D8': 'HISTORIAS',
        'D9': 'Historias laborales (UTL - ADMINISTRATIVAS)',
    },
    13000: {
        'D6': 'PRESIDENCIA',
        'D7': 'SECRETARÍA GENERAL',
        'D8': 'HISTORIAS LABORALES',
        'D9': '',
    },
    810: {
        'D6': 'DIRECCIÓN ADMINISTRATIVA',
        'D7': 'DIVISIÓN DE PERSONAL',
        'D8': 'HISTORIAS LABORALES',
        'D9': '',
    },
}

os.makedirs(OUTPUT_DIR, exist_ok=True)

for fname in os.listdir(TEMPLATE_DIR):
    if not fname.lower().endswith('_hoja_control.xlsx'):
        continue

    path_in = os.path.join(TEMPLATE_DIR, fname)
    nombre_archivo = fname.replace('_hoja_control.xlsx','')
    key = nombre_archivo.upper().strip()

    match = df.loc[df['nombre_up'] == key]
    if match.empty:
        print(f'⚠️ No encontré "{nombre_archivo}" en el maestro. Se salta.')
        continue

    cedula = match.iloc[0]['cedula']
    depe   = int(match.iloc[0]['depe'])
    info   = DEPE_MAP.get(depe)
    if info is None:
        print(f'⚠️ Valor de depe={depe} no mapeado para "{nombre_archivo}". Se salta.')
        continue

    wb = load_workbook(path_in)
    ws = wb[SHEET_NAME] if SHEET_NAME in wb.sheetnames else wb.active

    # ——— NUEVA LÍNEA: reemplazar título combinado A11:C11 ———
    ws['A11'] = "N° IDENTIFICACION:"
    # (la celda A11 está combinada con B11 y C11, así que basta con escribir en A11)

    # Relleno de datos
    ws['D10'] = nombre_archivo
    ws['D11'] = cedula
    ws['D6']  = info['D6']
    ws['D7']  = info['D7']
    ws['D8']  = info['D8']
    ws['D9']  = info['D9']

    path_out = os.path.join(OUTPUT_DIR, fname)
    wb.save(path_out)
    print(f'✅ Generado: {path_out}')